In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir_corr = f"datasets/friedman_correlated/many"
results_dir_tanh_corr = "results/regression/single_layer/tanh/friedman_correlated"

#model_names_tanh_nodewise_corr = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise"]#, "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]
model_names_tanh_nodewise_corr = ["Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise", "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]

tanh_fits_nodewise_corr = {}

files = sorted(f for f in os.listdir(data_dir_corr) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    

    tanh_fit_nodewise_corr = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh_corr,
        models=model_names_tanh_nodewise_corr,
        include_prior=False,
    )

    tanh_fits_nodewise_corr[base_config_name] = tanh_fit_nodewise_corr  # use clean key

In [ ]:
data_dir_corr = f"datasets/friedman_correlated"

files = sorted(f for f in os.listdir(data_dir_corr) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    
    tanh_fit_nodewise_corr = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh_corr,
        models=model_names_tanh_nodewise_corr,
        include_prior=False,
    )

    tanh_fits_nodewise_corr[base_config_name] = tanh_fit_nodewise_corr  # use clean key

In [ ]:
data_dir = f"datasets/friedman/many"
results_dir_tanh = "results/regression/single_layer/tanh/friedman"

#model_names_tanh_nodewise = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise"]#, "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]
model_names_tanh_nodewise = ["Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise", "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]

tanh_fits_nodewise = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    

    tanh_fit_nodewise = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_nodewise,
        include_prior=False,
    )

    tanh_fits_nodewise[base_config_name] = tanh_fit_nodewise  # use clean key

In [ ]:
data_dir = f"datasets/friedman"

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    
    tanh_fit_nodewise = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_nodewise,
        include_prior=False,
    )

    tanh_fits_nodewise[base_config_name] = tanh_fit_nodewise  # use clean key

## RMSE and CRPS

In [6]:
from sklearn.model_selection import train_test_split
from utils.generate_data import sample_gaussian_copula_uniform

def generate_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    np.random.seed(seed)
    X = np.random.uniform(0, 1, size=(N, D))
    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )
    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def generate_correlated_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    """
    Generate synthetic regression data for Bayesian neural network experiments.

    Parameters:
        N (int): Number of samples.
        D (int): Number of features.
        sigma (float): Noise level.
        test_size (float): Proportion for test split.
        seed (int): Random seed.
        standardize_y (bool): Whether to standardize the response variable.

    Returns:
        tuple: (X_train, X_test, y_train, y_test, y_mean, y_std) if standardize_y,
               else (X_train, X_test, y_train, y_test)
    """
    np.random.seed(seed)
    d = 10
    S_custom = np.eye(d)
    # Block 1 (vars 0..4): high Spearman, 0.7
    for i in range(0, 3):
        for j in range(i+1, 3):
            S_custom[i, j] = S_custom[j, i] = 0.8
    # Block 2 (vars 5..9): moderate Spearman, 0.4
    for i in range(5, 10):
        for j in range(i+1, 10):
            S_custom[i, j] = S_custom[j, i] = -0.5
    # Cross-block weaker, 0.15
    for i in range(0, 5):
        for j in range(5, 10):
            S_custom[i, j] = S_custom[j, i] = 0.15
    # A couple of bespoke pairs:
    S_custom[0, 9] = S_custom[9, 0] = 0.4
    S_custom[2, 7] = S_custom[7, 2] = 0.9  # very strong (will be projected if infeasible)
    S_custom[3, 4] = S_custom[4, 3] = -0.9  # very strong (will be projected if infeasible)
    S_custom[1, 6] = S_custom[6, 1] = -0.9  # very strong (will be projected if infeasible)

    U, _ = sample_gaussian_copula_uniform(n=10000, S=S_custom, random_state=123)
    #X = np.random.uniform(0, 1, size=(N, D))
    if N != U.shape[0]:
        idx = np.random.choice(U.shape[0], size=N, replace=False)
        X = U[idx, :]
    else:
        X = U

    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )

    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def make_large_eval_set(
    generator_fn,
    N_train,
    D,
    sigma,
    seed,
    n_eval=5000,
    standardize_y=True
):
    """
    Returns X_eval, y_eval (standardized if standardize_y=True), plus y_mean,y_std
    defined from the training split.
    """
    N_total = N_train + n_eval

    X_tr, X_te, y_tr, y_te, y_mean, y_std = generator_fn(
        N=N_total, D=D, sigma=sigma, test_size=n_eval / N_total, seed=seed,
        standardize_y=standardize_y, return_scale=True
    )
    # Now X_te has approx n_eval points (exact given test_size construction).
    return X_te, np.asarray(y_te).squeeze(), y_mean, y_std

In [7]:
def compute_rmse(
    seeds, all_seeds, models, all_fits, get_N_sigma, forward_pass,
    folder,
    n_eval=5000,
    D=10,
    standardize_y=True,
    # pass the correct generator functions
    gen_uncorr=None,
    gen_corr=None,
):
    """
    Evaluate on a large generated test set instead of the stored tiny X_test/y_test.
    Assumes model was trained on standardized y if standardize_y=True.
    """
    assert gen_corr is not None #and gen_uncorr is not None, "Pass both generator functions."

    posterior_means = []

    # choose generator based on folder name
    def choose_gen(folder):
        return gen_corr if "friedman_correlated" in folder else gen_uncorr

    for seed in seeds:
        N, sigma = get_N_sigma(seed, all_seeds)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'

        # Build large eval set consistent with training split standardization
        gen_fn = choose_gen(folder)
        X_test, y_test, y_mean, y_std = make_large_eval_set(
            generator_fn=gen_fn,
            N_train=N,
            D=D,
            sigma=sigma,
            seed=seed,
            n_eval=n_eval,
            standardize_y=standardize_y
        )

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                W1_samples = fit.stan_variable("W_1")           # (S, P, H)
                W2_samples = fit.stan_variable("W_L")           # (S, H, O)
                b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
                b2_samples = fit.stan_variable("output_bias")   # (S, O)

            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = W1_samples.shape[0]
            y_hats = np.zeros((S, y_test.shape[0]))
            for i in range(S):
                W1 = W1_samples[i]
                W2 = W2_samples[i]
                y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i]).squeeze()
                y_hats[i] = y_hat
                
            posterior_mean = y_hats.mean(axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test)**2))
            
            out_pm = {
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'n_eval': y_test.shape[0],
                'posterior_mean_rmse': posterior_mean_rmse,
                'posterior_mean_rmse_orig': posterior_mean_rmse * y_std,  # back to original y scale
            }
            
            posterior_means.append(out_pm)
            
    return pd.DataFrame(posterior_means)




In [8]:
from utils.sparsity import forward_pass_tanh
N100_seeds_corr = [1, 2, 3, 4, 5]
N200_seeds_corr = [6, 7, 8, 9, 10]
N500_seeds_corr = [11, 12, 13, 14, 15]
all_seeds_corr = [N100_seeds_corr, N200_seeds_corr, N500_seeds_corr]
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

def get_N_sigma(seed, all_seeds):
    if seed in all_seeds[0]:
        N=100
    elif seed in all_seeds[1]:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma


In [9]:
rmse_df_corr = compute_rmse(
        seeds=seeds,
        all_seeds=all_seeds_corr,
        models=model_names_tanh_nodewise_corr,
        all_fits=tanh_fits_nodewise_corr,
        get_N_sigma=get_N_sigma,
        forward_pass=forward_pass_tanh,
        folder="friedman_correlated",
        n_eval=1000,
        D=10,
        standardize_y=True,
        gen_uncorr=generate_Friedman_data_v2,
        gen_corr=generate_correlated_Friedman_data_v2,
    )

In [10]:
from utils.sparsity import forward_pass_tanh
N100_seeds = [1, 3, 4, 5, 6]
N200_seeds = [2, 7, 8, 9, 10]
N500_seeds = [11, 12, 13, 14, 15]
all_seeds = [N100_seeds, N200_seeds, N500_seeds]

In [11]:
rmse_df = compute_rmse(
        seeds=seeds,
        all_seeds=all_seeds,
        models=model_names_tanh_nodewise,
        all_fits=tanh_fits_nodewise,
        get_N_sigma=get_N_sigma,
        forward_pass=forward_pass_tanh,
        folder="friedman",
        n_eval=1000,
        D=10,
        standardize_y=True,
        gen_uncorr=generate_Friedman_data_v2,
        gen_corr=generate_correlated_Friedman_data_v2,
    )

In [12]:
def aggregate_over_seeds(df: pd.DataFrame, metric_col: str, agg_fun: str = "mean") -> pd.DataFrame:
    """
    Aggregate metric_col across seeds within each (N, sparsity, model, sigma).
    Returns a dataframe with columns:
      N, sparsity, model, sigma, n_seeds, center, lo, hi, sd, se
    where lo/hi depend on SHOW_BAND + BAND_KIND.
    """
    group_cols = ["N", "model"]
    if "sigma" in df.columns:
        group_cols = ["sigma"] + group_cols

    g = df.groupby(group_cols)[metric_col]

    out = g.agg(
        n_seeds="count",
        mean="mean",
        median="median",
        sd="std",
    ).reset_index()

    out["sd"] = out["sd"].fillna(0.0)
    out["se"] = np.where(out["n_seeds"] > 0, out["sd"] / np.sqrt(out["n_seeds"]), np.nan)

    if agg_fun == "median":
        out["center"] = out["median"]
    else:
        out["center"] = out["mean"]
    return out


In [13]:
rmse_df_corr_agg = aggregate_over_seeds(rmse_df_corr, 'posterior_mean_rmse_orig')

In [14]:
rmse_df_agg = aggregate_over_seeds(rmse_df, 'posterior_mean_rmse_orig')

In [ ]:
rmse_df_corr_agg.sort_values(by=["model", "N", "mean"]).round(3)

In [ ]:
rmse_df_agg.sort_values(by=["model", "N", "mean"]).round(3)

## CRPS

In [17]:
import numpy as np
import pandas as pd
from properscoring import crps_ensemble

def random_subset_idx(n, n_sub=None, seed=1):
    n_sub = n if n_sub is None else n_sub
    rng = np.random.default_rng(seed)
    return rng.choice(n, size=n_sub, replace=False)

def compute_crps_friedman(
    seeds, all_seeds, models, all_fits, get_N_sigma, forward_pass,
    folder,
    n_eval=5000,
    D=10,
    standardize_y=True,
    gen_uncorr=None,
    gen_corr=None,
    n_sub=1000,                 # subset of eval points for speed; set None for all
    subset_seed=123,
    sigma_name="sigma",
    use_orig_scale=True,        # report CRPS on original y scale
):
    """
    Computes CRPS for Friedman on a large generated eval set (n_eval).
    Uses posterior predictive draws y = mu + sigma * eps for each posterior draw.
    Returns one row per (seed, N, sigma, model) with seed-level summaries.
    """
    assert gen_corr is not None

    def choose_gen(folder):
        return gen_corr if "friedman_correlated" in folder else gen_uncorr

    rows = []

    for seed in seeds:
        N, sigma = get_N_sigma(seed, all_seeds)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'

        gen_fn = choose_gen(folder)
        X_test, y_test, y_mean, y_std = make_large_eval_set(
            generator_fn=gen_fn,
            N_train=N,
            D=D,
            sigma=sigma,
            seed=seed,
            n_eval=n_eval,
            standardize_y=standardize_y
        )

        y_test = np.asarray(y_test).reshape(-1)
        idx = random_subset_idx(len(y_test), n_sub=n_sub, seed=subset_seed + seed)

        y_sub = y_test[idx]
        # scale factor to original units (only matters if y is standardized)
        scale = (y_std if (standardize_y and use_orig_scale) else 1.0)

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                W1_samples = fit.stan_variable("W_1")
                W2_samples = fit.stan_variable("W_L")
                b1_samples = fit.stan_variable("hidden_bias")
                b2_samples = fit.stan_variable("output_bias")
                sig_s = np.asarray(fit.stan_variable(sigma_name)).reshape(-1)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = W1_samples.shape[0]
            sig_s = sig_s[:S]  # align just in case

            # Compute mu draws on the LARGE eval set, then subset columns
            mu_sub = np.zeros((S, len(idx)))  # (S, n_sub)
            for i in range(S):
                mu = forward_pass(
                    X_test,
                    W1_samples[i],
                    b1_samples[i][0],
                    W2_samples[i],
                    b2_samples[i]
                ).squeeze()
                mu_sub[i] = mu[idx]

            # Posterior predictive draws for y on the subset
            rng = np.random.default_rng(10_000 + seed)  # reproducible per seed
            eps = rng.standard_normal(size=mu_sub.shape)
            y_samps_sub = mu_sub + sig_s[:, None] * eps   # (S, n_sub)

            # CRPS is scale-equivariant: if you multiply y and samples by c, CRPS multiplies by c.
            crps_pointwise = crps_ensemble(y_sub * scale, (y_samps_sub * scale).T)  # (n_sub,)

            rows.append({
                "seed": seed,
                "N": N,
                "sigma": sigma,
                "model": model,
                "n_eval": int(n_eval),
                "n_sub": int(len(idx)),
                "crps_mean": float(np.mean(crps_pointwise)),
                "crps_median": float(np.median(crps_pointwise)),
                "crps_q25": float(np.quantile(crps_pointwise, 0.25)),
                "crps_q75": float(np.quantile(crps_pointwise, 0.75)),
            })

    return pd.DataFrame(rows)


In [26]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

abbr = {
    "Gaussian tanh": "Gauss",
    "Regularized Horseshoe tanh": "RHS",
    "Dirichlet Horseshoe tanh nodewise": "DHS",
    "Dirichlet Student T tanh nodewise": "DST",
    "Beta Horseshoe tanh nodewise": "BHS",
    "Beta Student T tanh nodewise": "BST"
}

def plot_friedman_crps_grouped_box(
    df_crps,
    model_order,
    N_order=(100, 200, 500),
    metric_col="crps_median",   # or "crps_mean"
    abbr=None,
    sizes_labels=None,
    figsize=(7, 2.8),
    title=None,
):
    if abbr is None:
        abbr = {m: m for m in model_order}
    labels = [abbr.get(m, m) for m in model_order]

    if sizes_labels is None:
        sizes_labels = [f"N={N}" for N in N_order]

    x = np.arange(len(model_order)) + 1
    offsets = np.array([-0.22, 0.00, 0.22])[:len(N_order)]
    width = 0.16
    colors = ["C0", "C1", "C2", "C3"][:len(N_order)]

    fig, ax = plt.subplots(figsize=figsize)

    for j, (N, col) in enumerate(zip(N_order, colors)):
        data_j = []
        for m in model_order:
            vals = df_crps.loc[(df_crps["model"] == m) & (df_crps["N"] == N), metric_col].dropna().values
            data_j.append(vals)

        pos_j = x + offsets[j]
        bp = ax.boxplot(
            data_j,
            positions=pos_j,
            widths=width,
            patch_artist=True,
            showfliers=False
        )

        for box in bp["boxes"]:
            box.set_facecolor(col)
            box.set_alpha(0.55)
            box.set_edgecolor("black")
        for k in ["whiskers", "caps", "medians"]:
            for line in bp[k]:
                line.set_color("black")
                line.set_linewidth(1.0)

    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_ylabel(metric_col.replace("_", " ").upper())
    ax.grid(axis="y", alpha=0.3)

    handles = [
        Patch(facecolor=colors[i], edgecolor="black", alpha=0.55, label=sizes_labels[i])
        for i in range(len(N_order))
    ]
    ax.legend(handles=handles, title="Train size", frameon=False)

    if title:
        ax.set_title(title)

    fig.tight_layout()
    plt.savefig("figures_for_use_in_paper/friedman_crps_corr_beta.pdf", bbox_inches="tight")
    plt.show()


In [19]:
df_crps_corr = compute_crps_friedman(
    seeds=seeds,
    all_seeds=all_seeds_corr,
    models=model_names_tanh_nodewise,
    all_fits=tanh_fits_nodewise_corr,              # same structure as in RMSE
    get_N_sigma=get_N_sigma,
    forward_pass=forward_pass_tanh,
    folder="friedman_correlated",                  # e.g. "friedman_correlated"
    n_eval=1000,
    D=10,
    standardize_y=True,
    gen_uncorr=generate_Friedman_data_v2,
    gen_corr=generate_correlated_Friedman_data_v2,
    n_sub=1000,                      # CRPS subset size
)


In [20]:
df_crps = compute_crps_friedman(
    seeds=seeds,
    all_seeds=all_seeds,
    models=model_names_tanh_nodewise,
    all_fits=tanh_fits_nodewise,              # same structure as in RMSE
    get_N_sigma=get_N_sigma,
    forward_pass=forward_pass_tanh,
    folder="friedman",                  # e.g. "friedman_correlated"
    n_eval=1000,
    D=10,
    standardize_y=True,
    gen_uncorr=generate_Friedman_data_v2,
    gen_corr=generate_correlated_Friedman_data_v2,
    n_sub=1000,                      # CRPS subset size
)

In [ ]:
plot_friedman_crps_grouped_box(
    df_crps=df_crps_corr,
    model_order=model_names_tanh_nodewise,
    N_order=(100, 200, 500),
    metric_col="crps_median",
    abbr=abbr,
    figsize=(4, 3),
    title=""
)

In [ ]:
plot_friedman_crps_grouped_box(
    df_crps=df_crps,
    model_order=model_names_tanh_nodewise,
    N_order=(100, 200, 500),
    metric_col="crps_median",
    abbr=abbr,
    figsize=(4, 3),
    title=""
)